## **Populating Algorhythm Database from Spotify API**

### **Authors**:

`Simón Correa Marín`

`Luis Felipe Ospina Giraldo`

### **Date**:

2025-05-29

### **Description**:

This notebook extracts musical data from the Spotify API using the `Spotipy` library in Python and populates the SQLite database designed for the **AlgoRythm** music recommendation system. It handles OAuth authentication, collects user profile information (liked songs, top tracks, and recently played songs), and fetches curated Spotify playlists such as "Top 50 Global" and "Today's Top Hits".

It also includes:

- Functions to safely insert and update user, artist, album, and track data.
- Ingestion of track metadata and user interaction history.
- Exporting database tables to CSV files for later use in analysis or model training.
- Deduplication logic when exporting CSVs.
- Integration with `dotenv` to manage Spotify credentials securely.

This notebook is a foundational step in building a hybrid recommendation system, ensuring the raw musical and behavioral data is cleanly structured and readily accessible for analysis and modeling in the AlgoRythm pipeline.


### **Libraries Packages**


In [1]:
# Install required libraries with uv and put them in the dev group
!uv add --group dev spotipy
!uv add --group dev python-dotenv

Resolved 155 packages in 0.55ms
Audited 149 packages in 0.03ms
Resolved 155 packages in 0.49ms
Audited 149 packages in 0.02ms
Resolved 155 packages in 0.49ms
Audited 149 packages in 0.02ms


### **Algorhythm Database Creation**


In [2]:
# Database setup and building script for SQLite
!python db/sqlite-setup.py

Database created at: ../../data/01_raw/db/algorhythm.db


### **Libraries**


In [3]:
# Common imports
import os
import pandas as pd

# DB and Spotify API setup
import sqlite3
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Load environment variables .env
import secrets
from dotenv import load_dotenv

# Other imports
import time
from datetime import datetime

### **Secrets and environment variables**


In [4]:
# Load environment variables from .env file
load_dotenv() 

SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")
SPOTIFY_REDIRECT_URI = os.getenv("SPOTIFY_REDIRECT_URI")

### **Spotipy Client Auth and DB connection**


In [5]:
# Create a Spotipy client with OAuth authentication

sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_CLIENT_SECRET,
        redirect_uri=SPOTIFY_REDIRECT_URI,
        scope="user-library-read user-read-recently-played user-top-read", # Scopes for reading user library and top tracks
    )
)

In [6]:
# Database connection
conn = sqlite3.connect("../../data/01_raw/db/algorhythm.db")
cursor = conn.cursor()

### **Getting Data**


In [7]:
# Random user data because we don't have real user data access 
locations = ["Colombia", "Mexico", "Argentina", "Chile", "Brazil", "USA", "Spain", "Germany"]
genders = ["male", "female"]
music_profiles = [
    "reggaeton", "urbano latino", "latin pop", "r&b", "bedroom pop", "country", "acoustic country",
    "rap", "trap latino", "soft pop", "pop", "melodic rap", "latin hip hop", "soundtrack", "edm",
    "argentine trap", "guaracha", "rage rap", "christmas", "indie", "dream pop", "salsa", "merengue",
    "moombahton", "hyperpop", "k-pop", "bachata"
]

In [8]:
# Create user profile with the Spotify API and real user_id
try:
    profile_data = sp.current_user()
    user_id = profile_data["id"]
except Exception as e:
    print(f"Could not get Spotify user profile: {e}")

age = secrets.randbelow(18) + 18  # [18, 35)
gender = secrets.choice(genders)
location = secrets.choice(locations)
profile = secrets.choice(music_profiles)

# Insert user if not already in the database
cursor.execute("SELECT 1 FROM User WHERE user_id = ?", (user_id,))
if cursor.fetchone() is None:
    cursor.execute(
        """
        INSERT INTO User (user_id, age, gender, location, music_profile)
        VALUES (?, ?, ?, ?, ?)
    """,
        (user_id, age, gender, location, profile),
    )
    print(f"Inserted new user: {user_id}")
else:
    print(f"User already exists: {user_id}")

conn.commit()

Inserted new user: 94aet5tklbier5yaxaofrdl0w


### **Function Definitions**


In [9]:
# Artist
def insert_artist(cursor, artist):
    cursor.execute(
        """
        INSERT OR IGNORE INTO Artist (artist_id, name, popularity, genres)
        VALUES (?, ?, ?, ?)
    """,
        (
            artist["id"],
            artist["name"],
            artist.get("popularity", 0),
            ", ".join(artist.get("genres", [])),
        ),
    )

In [10]:
# Album
def insert_album(cursor, sp, album, artist_id):
    album_details = sp.album(album["id"])
    cursor.execute(
        """
        INSERT OR IGNORE INTO Album (album_id, name, artist_id, release_date, popularity, genres)
        VALUES (?, ?, ?, ?, ?, ?)
    """,
        (
            album_details["id"],
            album_details["name"],
            artist_id,
            album_details.get("release_date", ""),
            album_details.get("popularity", 0),
            ", ".join(album_details.get("genres", [])),
        ),
    )

In [11]:
# Track
def insert_track(
    cursor,
    sp,
    track,
    album_id,
    artist_id,
    user_id,
    is_liked=False,
    is_recent=False,
    is_top=False,
    played_at=None,
):
    track_id = track.get("id")
    if not track_id:
        print(" Track without ID. Omiting...")
        return

    # Get artist genres
    genres = []
    try:
        artist_details = sp.artist(artist_id)
        genres = artist_details.get("genres", [])
    except Exception as e:
        print(f"Could not get genres for artist {artist_id}: {e}")

    release_date = track["album"].get("release_date", "") if track.get("album") else ""

    cursor.execute(
        """
        INSERT OR IGNORE INTO Track (track_id, name, artist_id, album_id, popularity, genres, release_date)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """,
        (
            track_id,
            track.get("name", "Unknown Track"),
            artist_id,
            album_id,
            track.get("popularity", 0),
            ", ".join(genres),
            release_date,
        ),
    )

    if not played_at:
        played_at = datetime.now().isoformat()

    cursor.execute(
        """
        INSERT OR IGNORE INTO UserTrackHistory (user_id, track_id, played_at, is_top, is_recent, is_liked)
        VALUES (?, ?, ?, ?, ?, ?)
    """,
        (user_id, track_id, played_at, int(is_top), int(is_recent), int(is_liked)),
    )

In [ ]:
# Top tracks
top_tracks = sp.current_user_top_tracks(limit=50, time_range="medium_term")["items"]
for track in top_tracks:
    artist = sp.artist(track["artists"][0]["id"])
    insert_artist(cursor, artist)
    insert_album(cursor, sp, track["album"], artist["id"])
    insert_track(cursor, sp, track, track["album"]["id"], artist["id"], user_id, is_top=True)

# Recent tracks
recent_tracks = sp.current_user_recently_played(limit=50)["items"]
for item in recent_tracks:
    track = item["track"]
    artist = sp.artist(track["artists"][0]["id"])
    insert_artist(cursor, artist)
    insert_album(cursor, sp, track["album"], artist["id"])
    insert_track(cursor, sp, track, track["album"]["id"], artist["id"], user_id, is_recent=True)

# Liked tracks (paginated) to avoid rate limits
limit = 50
offset = 0
while True:
    liked_page = sp.current_user_saved_tracks(limit=limit, offset=offset)["items"]
    if not liked_page:
        break
    for item in liked_page:
        track = item["track"]
        artist = sp.artist(track["artists"][0]["id"])
        insert_artist(cursor, artist)
        insert_album(cursor, sp, track["album"], artist["id"])
        insert_track(cursor, sp, track, track["album"]["id"], artist["id"], user_id, is_liked=True)
        time.sleep(0.1)  # Add delay to avoid rate limits
    offset += limit

conn.commit()

### **Saving data from DB into CSV files**


In [ ]:
# DB route
db_path = "../../data/01_raw/db/algorhythm.db"

# Tables to export
tables = ["User", "Artist", "Album", "Track", "UserTrackHistory"]
output_folder = os.path.dirname("../../data/01_raw/csv/")

# Ensure the output directory exists
os.makedirs(output_folder, exist_ok=True)

# Export tables to CSV with exception handling
for table in tables:
    try:
        df = pd.read_sql_query(f"SELECT * FROM [{table}]", sqlite3.connect(db_path))
        file_path = os.path.join(output_folder, f"{table}.csv")
        df.to_csv(file_path, index=False)
        print(f"{table}.csv saved to: {file_path}")
    except Exception as e:
        print(f"Failed to export table {table}: {e}")

# Load CSVs into a dictionary with exception handling
df_dict = {}
for file in os.listdir(output_folder):
    if file.endswith(".csv"):
        name = file[:-4]  # remove ".csv"
        file_path = os.path.join(output_folder, file)
        try:
            df = pd.read_csv(file_path)
            df_dict[name] = df
            print(f"Loaded: {name} ({df.shape[0]} rows)")
        except Exception as e:
            print(f"Failed to load {file}: {e}")

# Add DataFrames to globals
globals().update(df_dict)

### **Load Data**


In [13]:
# Load dataframes from CSV files into a dictionary
folder_path = "../../data/01_raw/csv/"
dataframes = {}

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        name = os.path.splitext(filename)[0]
        file_path = os.path.join(folder_path, filename)
        try:
            df = pd.read_csv(file_path)
            dataframes[name] = df
            print(f"Dataframe: {name} ({df.shape[0]} rows)")
        except Exception as e:
            print(f"Error loading {filename}: {e}")

Dataframe: Artist (748 rows)
Dataframe: Album (1772 rows)
Dataframe: Track (2468 rows)
Dataframe: UserTrackHistory (3052 rows)
Dataframe: User (1 rows)
Dataframe: ChartTrack (473 rows)


In [14]:
# Display the first rows of each DataFrame
for name, df in dataframes.items():
    print(f"{name} DataFrame:")
    display(df.head(3))

Artist DataFrame:


,artist_id,name,genres,popularity
0,4tuJ0bMpJh08umKkEXKUI5,Gracie Abrams,NaN,87
1,2sSGPbdZJkaSE2AbcGOACx,The Marías,bedroom pop,86
2,4Uc8Dsxct0oMqx0P6i60ea,Conan Gray,NaN,80


Album DataFrame:


,album_id,name,artist_id,release_date,popularity,genres
0,4XXTsu7r9865VvXdvF2iQP,The Secret of Us,4tuJ0bMpJh08umKkEXKUI5,2024-06-20,0,NaN
1,56bdWeO40o3WfAD2Lja4dl,The Secret of Us,4tuJ0bMpJh08umKkEXKUI5,2024-06-21,0,NaN
2,1Mo4aZ8pdj6L1jx8zSwJnt,THE TORTURED POETS DEPARTMENT,06HL4z0CvFAxyc27GXpf02,2024-04-18,0,NaN


Track DataFrame:


...

,user_id,track_id,played_at,is_top_track,is_recent_play,is_liked
0,94aet5tklbier5yaxaofrdl0w,6nN8W5zHOii0P61I8eSdR3,2025-05-28T07:20:47.132908,1.0,0.0,0
1,94aet5tklbier5yaxaofrdl0w,51rfRCiUSvxXlCSCfIztBy,2025-05-28T07:20:47.270594,1.0,0.0,0
2,94aet5tklbier5yaxaofrdl0w,5wbg8kepMFoMzHOEuxiI0q,2025-05-28T07:20:47.386019,1.0,0.0,0


User DataFrame:


,user_id,age,gender,location,music_profile
0,94aet5tklbier5yaxaofrdl0w,21,male,Colombia,"reggaeton, country, urbano latino, latin pop, ..."


ChartTrack DataFrame:


,chart_id,track_id,name,artist_id,album_id,popularity,genres,chart_name,position,added_at
0,4Jb4PDWREzNnbZcOHPcZPy,5IZXB5IKAD2qlvTPJYDCFB,I Had Some Help (Feat. Morgan Wallen),246dkjvS1zLTtiykXe5h60,4BbsHmXEghoPPevQjPnHXx,88,NaN,Today's Top Hits,1,2024-10-23T15:33:22Z
1,4Jb4PDWREzNnbZcOHPcZPy,2uqYupMHANxnwgeiXTZXzd,Austin (Boots Stop Workin'),7Ez6lTtSMjMf2YSYpukP1I,40HsqPqeSR9Xe3IyAJWr6e,88,NaN,Today's Top Hits,2,2024-05-09T22:50:38Z
2,4Jb4PDWREzNnbZcOHPcZPy,3Rfre3qkrhwdZZ7dyznwbN,Lonely Road (with Jelly Roll),6TIYQ3jFPwQSRmorSezPxX,4tU0FNnuiBD1P6IRTARHww,79,NaN,Today's Top Hits,3,2024-10-23T14:37:11Z


## **Chart Data to make recommendations**


In [15]:
# Function to insert tracks from the chart into the database
def insert_chart_track(
    cursor, track, chart_id, chart_name, position, added_at=None
):
    genres = []
    try:
        artist = track["artists"][0]
        artist_id = artist["id"]
        artist_name = artist["name"]
        artist_details = sp.artist(artist_id)
        genres = artist_details.get("genres", [])
    except Exception as e:
        print(f"Could not get genres for artist {artist_id}: {e}")
        artist_name = artist.get("name", "Unknown Artist") if 'artist' in locals() else "Unknown Artist"
        artist_id = artist.get("id", None) if 'artist' in locals() else None

    try:
        album = track["album"]
        album_id = album.get("id")
        album_name = album.get("name", "Unknown Album")
    except Exception:
        album_id = None
        album_name = "Unknown Album"

    cursor.execute(
        """
        INSERT OR IGNORE INTO ChartTrack (
            chart_id, track_id, name, artist_id, artist_name, album_id, album_name, popularity, genres, chart_name, position, added_at
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """,
        (
            chart_id,
            track.get("id"),
            track.get("name", "Unknown Track"),
            artist_id,
            artist_name,
            album_id,
            album_name,
            track.get("popularity", 0),
            ", ".join(genres),
            chart_name,
            position,
            added_at,
        ),
    )

### **Searching dynamic data - Charts Updated Daily**


In [16]:
# Search for a playlist by name and return its ID
def search_playlist_id(sp, query, limit=10):
    results = sp.search(q=query, type="playlist", limit=limit)
    items = results.get("playlists", {}).get("items", [])
    for playlist in items:
        if playlist is not None and playlist.get("public", False):
            return {"name": playlist["name"], "id": playlist["id"]}
    return None

# Function to get top genre and daily playlists
def get_top_genre_and_daily_playlists(sp):
    top_playlists = {
        "Today's Top Hits": "Today's Top Hits",
        "Top 50 Global": "Top 50 Global",
    }
    
    countries = ["Colombia", "Mexico", "Argentina", "Chile", "Brazil", "USA", "Spain", "Germany"]

    for country in countries:
        top_playlists[f"Top 50 {country}"] = f"Top 50 {country}"

    results = {}

    for key, query in top_playlists.items():
        playlist = search_playlist_id(sp, query)
        results[key] = playlist["id"] if playlist else None

    return results

# Run the function and display results
playlist_ids = get_top_genre_and_daily_playlists(sp)
playlist_ids

{"Today's Top Hits": '5iwkYfnHAGMEFLiHFFGnP4',
 'Top 50 Global': '3NJmFR02JQRYIB7COYckSR',
 'Top 50 Colombia': '6h6uzoRBXnkjeoEjwiX27R',
 'Top 50 Mexico': '3NJmFR02JQRYIB7COYckSR',
 'Top 50 Argentina': '3NJmFR02JQRYIB7COYckSR',
 'Top 50 Chile': '6lS871B4ot8ALAb0axxRTX',
 'Top 50 Brazil': '2yf15hiBRKPKPQlJNfENUE',
 'Top 50 USA': '0fMcmgDViWL2zZgtzTrgLa',
 'Top 50 Spain': '7ki8VipxMygh7Y8ZdPRvmX',
 'Top 50 Germany': '54AhsdZbmVc5ieL6mSKrbN'}

In [ ]:
# For each playlist in playlist_ids, insert all tracks into ChartTrack with added_at and pagination
for chart_name, chart_id in playlist_ids.items():
    if not chart_id:
        continue
    try:
        limit = 100
        offset = 0
        position = 1
        while True:
            response = sp.playlist_items(chart_id, limit=limit, offset=offset)
            items = response['items']
            for item in items:
                track = item['track']
                # Get artist and album IDs for insertion into Artist/Album/Track tables
                artist_id = track['artists'][0]['id']
                album_id = track['album']['id']
                added_at = item.get('added_at', None)
                # Insert artist and album if not present
                insert_artist(cursor, sp.artist(artist_id))
                insert_album(cursor, sp, track['album'], artist_id)
                # Insert track (user_id=None for chart tracks)
                insert_track(cursor, sp, track, album_id, artist_id, user_id=None)
                # Insert into ChartTrack (all info extracted inside function)
                insert_chart_track(cursor, track, chart_id, chart_name, position, added_at)
                position += 1
                time.sleep(0.2)
            if response['next'] is None:
                break
            offset += limit
    except Exception as e:
        print(f"Error processing playlist {chart_name} ({chart_id}): {e}")

conn.commit()

### **Update CSV Files**


In [17]:
# Dictionary: table → primary keys
tables = {
    "User": ["user_id"],
    "Artist": ["artist_id"],
    "Album": ["album_id"],
    "Track": ["track_id"],
    "UserTrackHistory": ["user_id", "track_id", "played_at"],
    "ChartTrack": ["chart_id", "track_id"],
}


# Folder to save the CSVs
output_folder = "../../data/01_raw/csv/"
os.makedirs(output_folder, exist_ok=True)

# Export each table to CSV, avoiding duplicates
for table, primary_keys in tables.items():
    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    file_path = os.path.join(output_folder, f"{table}.csv")

    if os.path.exists(file_path):
        existing_df = pd.read_csv(file_path)

        # Concatenate and drop duplicates based on primary keys
        combined_df = pd.concat([existing_df, df], ignore_index=True)
        combined_df.drop_duplicates(subset=primary_keys, keep="last", inplace=True)
    else:
        combined_df = df

    # Save the updated file
    combined_df.to_csv(file_path, index=False)
    print(f"{table}.csv updated at: {file_path}")

User.csv updated at: ../../data/01_raw/csv/User.csv
Artist.csv updated at: ../../data/01_raw/csv/Artist.csv
Album.csv updated at: ../../data/01_raw/csv/Album.csv
Track.csv updated at: ../../data/01_raw/csv/Track.csv
UserTrackHistory.csv updated at: ../../data/01_raw/csv/UserTrackHistory.csv
ChartTrack.csv updated at: ../../data/01_raw/csv/ChartTrack.csv


In [18]:
# Close the database connection after all inserts
conn.close()

In [21]:
# Folder path for CSV files
folder_path = "../../data/01_raw/csv/"

dataframes = {}

# Load CSV files from the specified folder into a dictionary of DataFrames
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        name = os.path.splitext(filename)[0]
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        dataframes[name] = df
        print(f"DataFrame: {name} ({df.shape[0]} rows)")

DataFrame: Artist (748 rows)
DataFrame: Album (1717 rows)
DataFrame: Track (2416 rows)
DataFrame: UserTrackHistory (3052 rows)
DataFrame: User (1 rows)
DataFrame: ChartTrack (473 rows)


In [22]:
# Display the first rows of each DataFrame
for name, df in dataframes.items():
    print(f"{name} DataFrame:")
    display(df.head(3))

Artist DataFrame:


,artist_id,name,genres,popularity
0,4tuJ0bMpJh08umKkEXKUI5,Gracie Abrams,NaN,87
1,2sSGPbdZJkaSE2AbcGOACx,The Marías,bedroom pop,86
2,4Uc8Dsxct0oMqx0P6i60ea,Conan Gray,NaN,80


Album DataFrame:


,album_id,name,artist_id,release_date,popularity,genres
0,4XXTsu7r9865VvXdvF2iQP,The Secret of Us,4tuJ0bMpJh08umKkEXKUI5,2024-06-20,0,NaN
1,56bdWeO40o3WfAD2Lja4dl,The Secret of Us,4tuJ0bMpJh08umKkEXKUI5,2024-06-21,0,NaN
2,1Mo4aZ8pdj6L1jx8zSwJnt,THE TORTURED POETS DEPARTMENT,06HL4z0CvFAxyc27GXpf02,2024-04-18,0,NaN


Track DataFrame:


...

,user_id,track_id,played_at,is_top_track,is_recent_play,is_liked,is_top,is_recent
0,94aet5tklbier5yaxaofrdl0w,6nN8W5zHOii0P61I8eSdR3,2025-05-28T07:20:47.132908,1.0,0.0,0,NaN,NaN
1,94aet5tklbier5yaxaofrdl0w,51rfRCiUSvxXlCSCfIztBy,2025-05-28T07:20:47.270594,1.0,0.0,0,NaN,NaN
2,94aet5tklbier5yaxaofrdl0w,5wbg8kepMFoMzHOEuxiI0q,2025-05-28T07:20:47.386019,1.0,0.0,0,NaN,NaN


User DataFrame:


,user_id,age,gender,location,music_profile
0,94aet5tklbier5yaxaofrdl0w,21,male,Colombia,"reggaeton, country, urbano latino, latin pop, ..."


ChartTrack DataFrame:


,chart_id,track_id,name,artist_id,album_id,popularity,genres,chart_name,position,added_at,artist_name,album_name
0,4Jb4PDWREzNnbZcOHPcZPy,5IZXB5IKAD2qlvTPJYDCFB,I Had Some Help (Feat. Morgan Wallen),246dkjvS1zLTtiykXe5h60,4BbsHmXEghoPPevQjPnHXx,88,NaN,Today's Top Hits,1,2024-10-23T15:33:22Z,NaN,NaN
1,4Jb4PDWREzNnbZcOHPcZPy,2uqYupMHANxnwgeiXTZXzd,Austin (Boots Stop Workin'),7Ez6lTtSMjMf2YSYpukP1I,40HsqPqeSR9Xe3IyAJWr6e,88,NaN,Today's Top Hits,2,2024-05-09T22:50:38Z,NaN,NaN
2,4Jb4PDWREzNnbZcOHPcZPy,3Rfre3qkrhwdZZ7dyznwbN,Lonely Road (with Jelly Roll),6TIYQ3jFPwQSRmorSezPxX,4tU0FNnuiBD1P6IRTARHww,79,NaN,Today's Top Hits,3,2024-10-23T14:37:11Z,NaN,NaN
